In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm, uniform
from kan import *
import torch
from scipy.stats import multivariate_normal
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [ ]:
plt.rcParams.update({
    'font.size': 12,               # Font size
    'axes.labelsize': 15,          # Label size
    'axes.titlesize': 15,          # Title size
    'axes.linewidth': 1.2,         # Axis line width
    'xtick.labelsize': 10,         # Size of x-axis tick labels
    'ytick.labelsize': 10,         # Size of y-axis tick labels
    'xtick.major.size': 6,         # Length of major ticks on the x-axis
    'ytick.major.size': 6,         # Length of major ticks on the y-axis
    'xtick.major.width': 1.0,      # Width of major ticks on the x-axis
    'ytick.major.width': 1.0,      # Width of major ticks on the y-axis
    'figure.dpi': 300,             # Image resolution
    'savefig.dpi': 300,            # Resolution for saving figures
    'figure.figsize': (12, 5),     # Figure size
})

## Scenario B1

#### $I_{u, \mathbf{x}} \mid \, Y_{\mathbf{x}} > u\sim \text{Bern}\{m_{\text{B1}}(\mathbf{x}; u)\}$

In [ ]:
def MC_B1(M, n):
    x_min, x_max = 0, 1
    y_min, y_max = 0, 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    u_quantile = 0.95
    probabilities_all = [] 
    
    for i in range(M):
        np.random.seed(i) 
        
        # X1 and X2 ~ U(0,1)
        X1 = uniform.rvs(loc=0, scale=1, size=n)
        X2 = uniform.rvs(loc=0, scale=1, size=n)

        # Y_X ~ Unit Fréchet
        U = np.random.uniform(0, 1, size=n)
        Y_X = 1 / -np.log(n / (n + 1) * U)

        u = np.quantile(Y_X, u_quantile)
        mean = [np.exp(-u), np.exp(-u)]
        cov = [[1, 0], [0, 1]] 
        mvnorm = multivariate_normal(mean, cov)
        Phi_X = np.array([mvnorm.cdf([x1, x2]) for x1, x2 in zip(X1, X2)])
        mask = Y_X > u
        I = np.zeros(n, dtype=int)
        I[mask] = np.random.binomial(1, Phi_X[mask])

        # Dataframe
        df = pd.DataFrame({'X1': X1, 'X2': X2, 'Y': Y_X, 'I': I})
        df = df.sort_values(by=['X1', 'X2'], ascending=[True, True])
        df_sim = df[df['Y'] > u]

        # Neural inputs
        x_train_2d = np.zeros((df_sim.shape[0], 2))

        x_train_2d[:, 0] = df_sim['X1']
        x_train_2d[:, 1] = df_sim['X2']
        y_train_2d = df_sim['I'].values

        x_train_tensor = torch.tensor(x_train_2d, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train_2d, dtype=torch.long)

        model = KAN(width=[2, 2], grid=2, k=2)

        dataset = {
            'train_input': x_train_tensor,
            'train_label': y_train_tensor,
            'test_input': x_train_tensor,
            'test_label': y_train_tensor
        }

        model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss())

        grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

        with torch.no_grad():
            Z = model(grid_tensor)
            Z = torch.softmax(Z, dim=1)
            probabilities = Z[:, 1].reshape(xx.shape)

        probabilities_all.append(probabilities)

    return probabilities_all, xx, yy

In [ ]:
probabilities_all, xx, yy = MC_B1(500, 15000)

In [ ]:
probabilities_all = np.array(probabilities_all)
monte_carlo_mean_surface = np.mean(probabilities_all, axis=0)

def bivariate_cdf(x, y, mean, cov):
    pos = np.dstack((x, y))
    return multivariate_normal.cdf(pos, mean=mean, cov=cov)

cdf_values = bivariate_cdf(xx, yy, [0, 0], [[1, 0], [0, 1]] )

squared_difference = (monte_carlo_mean_surface - cdf_values) ** 2

integral_y = np.trapz(squared_difference, x=yy[:, 0], axis=0)
mise = np.trapz(integral_y, x=xx[0, :])

print('MISE: ' + str(mise))

fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title('MC Mean KANE POC Surface (B1)')

ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, cdf_values, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title('True POC Surface (B1)')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position()  
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 

plt.suptitle('Scenarios B1 and B2 (n = 15 000)', fontsize=18, fontweight='bold')

plt.tight_layout()
plt.show()

## Scenario B2

#### $I_{u, \mathbf{x}} \mid \, Y_{\mathbf{x}} > u\sim \text{Bern}\{m_{\text{B2}}(\mathbf{x}; u)\}$

In [ ]:
def MC_B2(M, n):
    def f(x, y):
        return 0.5 + 0.4 *np.exp(- x) * np.cos(2 * np.pi * y) + 1/u**2

    x_min, x_max = 0, 1
    y_min, y_max = 0, 1
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))

    u_quantile = 0.95
    probabilities_all = [] 
    
    for i in range(M):
        np.random.seed(i) 
        
        # X1 and X2 ~ U(0,1)
        X1 = uniform.rvs(loc=0, scale=1, size=n)
        X2 = uniform.rvs(loc=0, scale=1, size=n)

        # Y_X ~ Unit Fréchet
        U = np.random.uniform(0, 1, size=n)
        Y_X = 1 / -np.log(n / (n + 1) * U)
        
        u = np.quantile(Y_X, u_quantile)
        f_X = np.array([f(x1, x2) for x1, x2 in zip(X1, X2)])
        mask = Y_X > u
        I = np.zeros(n, dtype=int)
        I[mask] = np.random.binomial(1, f_X[mask])

        # Dataframe
        df = pd.DataFrame({'X1': X1, 'X2': X2, 'Y': Y_X, 'I': I})
        df = df.sort_values(by=['X1', 'X2'], ascending=[True, True])
        df_sim = df[df['Y'] > u]

        # Neural inputs
        x_train_2d = np.zeros((df_sim.shape[0], 2))

        x_train_2d[:, 0] = df_sim['X1']
        x_train_2d[:, 1] = df_sim['X2']
        y_train_2d = df_sim['I'].values

        x_train_tensor = torch.tensor(x_train_2d, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train_2d, dtype=torch.long)

        model = KAN(width=[2, 2], grid=2, k=3)

        dataset = {
            'train_input': x_train_tensor,
            'train_label': y_train_tensor,
            'test_input': x_train_tensor,
            'test_label': y_train_tensor
        }

        model.fit(dataset, opt="LBFGS", steps=100, loss_fn=torch.nn.CrossEntropyLoss())

        grid_tensor = torch.tensor(np.c_[xx.ravel(), yy.ravel()], dtype=torch.float32)

        with torch.no_grad():
            Z = model(grid_tensor)
            Z = torch.softmax(Z, dim=1)
            probabilities = Z[:, 1].reshape(xx.shape)

        probabilities_all.append(probabilities)

    return probabilities_all, xx, yy

In [ ]:
probabilities_all_2, xx, yy = MC_B2(500, 15000)

In [ ]:
def f(x, y):
    return 0.5 + 0.4 * np.exp(- x) * np.cos(2 * np.pi * y)

probabilities_all_2 = np.array(probabilities_all_2)
monte_carlo_mean_surface_2 = np.mean(probabilities_all_2, axis=0)

f_values = f(xx, yy)

squared_difference_2 = (monte_carlo_mean_surface_2 - f_values) ** 2

integral_y_2 = np.trapz(squared_difference_2, x=yy[:, 0], axis=0)
mise_2 = np.trapz(integral_y_2, x=xx[0, :])

print('MISE: ' + str(mise_2))

fig = plt.figure(figsize=(12, 5))
gs = gridspec.GridSpec(1, 3, width_ratios=[1, 1, 0.05], wspace=0.3)

ax1 = fig.add_subplot(gs[0])
im1 = ax1.pcolor(xx, yy, monte_carlo_mean_surface_2, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax1.set_xlabel(r'$x_1$', fontsize=16)
ax1.set_ylabel(r'$x_2$', fontsize=16)
ax1.set_title('MC Mean KANE POC Surface (B2)')

ax2 = fig.add_subplot(gs[1])
im2 = ax2.pcolor(xx, yy, f_values, shading='auto', cmap='RdBu_r', vmin=0, vmax=1)
ax2.set_xlabel(r'$x_1$', fontsize=16)
ax2.set_ylabel(r'$x_2$', fontsize=16)
ax2.set_title('True POC Surface (B2)')

cbar_ax = fig.add_subplot(gs[2])
fig.colorbar(im2, cax=cbar_ax)

pos = cbar_ax.get_position()  
cbar_ax.set_position([pos.x0 - 0.02, pos.y0, pos.width, pos.height]) 

plt.tight_layout()
plt.show()